In [11]:
## 1- Intalar a biblioteca google-genai

%pip -q install google-genai

In [9]:
## 2- Configurar a API do google-genai

import os ##Importa o Módulo os
from google.colab import userdata ##Importa o Módulo userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY') #Armazena a APi como uma variável

In [13]:
## 3- Configura o cliente da SDK do Gemini

from google import genai #Importa o SDK

client = genai.Client() #Cria uma instância do cliente

MODEL_ID = "gemini-2.0-flash" #Define o modelo

In [14]:
### 4 - Instala o Google-ADK

%pip install google-adk

In [1]:
# 5 Importar bibliotecas do ADK

from google.adk.agents import Agent # Permite criar agente personalizado.
from google.adk.runners import Runner # Permite controlar o fluxo da conversa usuário/IA.
from google.adk.sessions import InMemorySessionService # Cria sessões em memória para manter o contexto.
from google.adk.tools import google_search # Permite consulta na web via google.
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date # Para capturar data atual
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore") # Para suprimir avisos e manter o notebook limpo.

In [119]:
# 6 Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
import uuid
import nest_asyncio
import asyncio

nest_asyncio.apply()

def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = asyncio.run(session_service.create_session(app_name=agent.name, user_id="user1"))
    # Cria a sessão com ID
    session_id = str(uuid.uuid4())
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in asyncio.run(runner.run(user_id="user1", session_id = session_id, new_message=content)):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [120]:
# 7 Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [121]:
##########################################
# --- Agente 1: Diagnóstico--- #
##########################################

agente_diagnostico = Agent(
    name = "Diagnóstico",
    description = "Agente que faz o diagnóstico de um problema.",
    model = MODEL_ID,
    instruction = """
    Você é um consultor experiente em gestão empresarial. Sua tarefa é interpretar o texto enviado por um gestor que descreve um desafio real em seu trabalho.

    Extraia e retorne de forma clara:
    - Tipo de desafio (ex: produtividade, comunicação, priorização, sobrecarga, etc.)
    - Área afetada (ex: projetos, processos, equipe, tecnologia, etc.)
    - Grau de urgência (baixa, média ou alta)

Texto do gestor:
"""
)


In [122]:
##########################################
# --- Agente 2: Recomendação--- #
##########################################

agente_recomendacao = Agent(
    name = "Recomendação",
    description = "Agente que faz recomendações para resolver um problema.",
    model = "gemini-2.0-flash-thinking-exp",
    instruction = """
    Com base no diagnóstico a seguir, recomende 3 ferramentas ou métodos ou frameworks que sejam práticos para o gestor utilizar.
    Você deve considerar o tipo de desafio, a área afetada e o grau de urgência.
    Para cada recomendação, forneça uma breve descrição de como a ferramenta pode ajudar a resolver o problema e por que ela é adequada para o contexto do gestor.
    Categorize as recomendações com base na complexidade de implementação: baixa, média ou alta.
    Use a seguinte estrutura para cada recomendação:
    - Tipo de desafio:
    - Nome da ferramenta ou método:
    - Descrição:
    - Saídas esperadas: (ex: aumento de produtividade, redução de custos, melhoria na comunicação, etc.)
    - Complexidade de implementação: (baixa, média ou alta)

    Diagnóstico:
    {{input}}

    """
)

In [123]:
##########################################
# --- Agente 3: Justificativa --- #
##########################################

agente_justificativa = Agent(
    name = "Justificativa",
    description = "Agente que cria a saída para resolver um problema.",
    model = MODEL_ID,
    tools = [google_search],
    instruction = """
    Você é um consultor especializado em gestão organizacional. Com base no diagnóstico a seguir, forneça **3 recomendações** de ferramentas ou métodos que possam auxiliar o gestor, organizadas por ordem crescente de complexidade de implementação (baixa, média, alta).

Para cada recomendação, apresente as seguintes informações:

- Nome da ferramenta ou método e autor:
- Descrição:
- Tipo de desafio: (ex: aumento de produtividade, redução de custos, melhoria na comunicação, etc.)
- Complexidade de implementação: (baixa, média ou alta)
- Link para aprofundamento: (Você deve apresentar um link relevante pesquisado no google_search)

**Diagnóstico:**
{{input}}

Apresente as informações de forma clara e objetiva.
    """
)

In [124]:
##########################################
# --- Fluxo de Orquestração --- #
##########################################

#função que orquestra o fluxo de trabalho

def recomendar_ferramenta(texto_desafio):
    # Chama o agente de diagnóstico
    resposta_diagnostico = call_agent(agente_diagnostico, texto_desafio)
    # Formata resposta do agente de diagnóstico
    to_markdown(F"###🔍 Diagnóstico:\n{resposta_diagnostico}")

    # Chama o agente de recomendação
    resposta_recomendacao = call_agent(agente_recomendacao, resposta_diagnostico)
    # Formata resposta do agente de recomendação
    to_markdown(F"### ✅ Recomendação:\n{resposta_recomendacao}")

    # Chama o agente de justificativa
    entrada_justificativa = f"{resposta_diagnostico.strip()}\n\n{resposta_recomendacao.strip()}"
    resposta_justificativa = call_agent(agente_justificativa, entrada_justificativa)
    to_markdown(F"### 📝 Justificativa:\n{resposta_justificativa}")

In [125]:
# 8 - Importa bibliotecas de interface gráfica
import ipywidgets as widgets # Para criar widgets interativos
from IPython.display import display # Para exibir widgets no Colab

In [126]:

# 9 - Cria um widget de entrada de texto para o usuário

entrada_usuario = widgets.Textarea(
    description='Desafio:',
    placeholder='Descreva o desafio que você está enfrentando...\nForneça informações como: área, tamanho da equipe, porte da empresa e etc...',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
)

In [127]:
#10 - Cria um botão para enviar a mensagem

botao_enviar = widgets.Button(
    description='Enviar',
    button_style='success',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    tolltip='Executar agente de recomendação'
)


In [128]:
# 12 - Função para o logica do botão
def ao_clicar_botao(b):
    # Captura o texto do usuário
    texto_desafio = entrada_usuario.value
    # Cria uma condição para verificar se o campo de entrada está vazio
    if not texto_desafio.strip():
        # Exibe um aviso se o campo estiver vazio
        display(Markdown("### ⚠️ Por favor, descreva o desafio que você está enfrentando."))
    else:
        recomendar_ferramenta(texto_desafio)
    # Limpa o campo de entrada após o envio
    entrada_usuario.value = ''

In [129]:
#13 Conecta o botão à função
botao_enviar.on_click(ao_clicar_botao)
# 14 - Exibe os widgets na tela
display(entrada_usuario, botao_enviar)

Textarea(value='', description='Desafio:', layout=Layout(height='200px', width='100%'), placeholder='Descreva …

Button(button_style='success', description='Enviar', layout=Layout(width='100%'), style=ButtonStyle())

Exception in thread Thread-22 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 98, in run_until_complete
    return f.result()
           ^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.11/asyncio/tasks.py", line 277, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/usr/

TypeError: 'NoneType' object is not iterable